# Setup

In [ ]:
!pip install -U matplotlib
!pip install scanpy
!pip install numpy
!pip install pandas
!pip install AnnData
!pip install wishbone_dev
!pip install leidenalg

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import anndata as ad

# Blattman E.coli data preprocessing

In [ ]:
blattman_ecoli = sc.read('/Blattman data/GSM4192509_394A_R2_trimmed_v4_v11_threshold_1_unfiltered_gene_matrix.txt', delimiter = '\t')
blattman_ecoli

AnnData object with n_obs × n_vars = 29548 × 2423

In [ ]:
sc.pp.calculate_qc_metrics(blattman_ecoli, inplace= True)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_qc.py:422: RuntimeWarning: invalid value encountered in true_divide
  return values / sums[:, None]


In [ ]:
dense2 = blattman_ecoli.copy()

In [ ]:
dense2 = dense2[:,:2418]
sc.pp.filter_cells(dense2,min_counts=15)
sc.pp.filter_cells(dense2,max_counts = 2000)
sc.pp.filter_genes(dense2,min_cells = 5)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_simple.py:138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_counts'] = number


In [ ]:
dense2

AnnData object with n_obs × n_vars = 12255 × 1983
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'n_counts'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'

In [ ]:
sc.pp.normalize_total(dense2, inplace = True)
dense2.obs['cell_sum']=dense2.X.sum(axis=1)
dense2.obs['cell_sum']

394A_bc1_10_bc2_10_bc3_79_none_0       201.000031
394A_bc1_10_bc2_11_bc3_7_GFP_1         200.999985
394A_bc1_10_bc2_12_bc3_83_GFP_3        201.000015
394A_bc1_10_bc2_15_bc3_43_none_5       201.000000
394A_bc1_10_bc2_15_bc3_7_GFP_6         201.000000
                                          ...    
394A_bc1_9_bc2_95_bc3_37_none_29540    201.000000
394A_bc1_9_bc2_95_bc3_43_GFP_29541     201.000031
394A_bc1_9_bc2_9_bc3_36_RFP_29544      201.000000
394A_bc1_9_bc2_9_bc3_50_GFP_29545      201.000031
394A_bc1_9_bc2_9_bc3_84_none_29547     200.999985
Name: cell_sum, Length: 12255, dtype: float32

In [ ]:
sc.pp.log1p(dense2)

In [ ]:
#sc.pp.regress_out(dense2,['total_counts'])
sc.pp.scale(dense2,max_value=10)

In [ ]:
express = []
for i in dense2.obs_names:
    if 'none' in i:
        express.append('none')
    elif 'RFP' in i:
        express.append('RFP')
    elif 'GFP' in i:
        express.append('GFP')
dense2.obs['express']= express
dense2.obs  

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,n_counts,cell_sum,express
394A_bc1_10_bc2_10_bc3_79_none_0,28,3.367296,202.0,5.313206,100.000000,100.0,100.0,100.0,202.0,201.000031,none
394A_bc1_10_bc2_11_bc3_7_GFP_1,40,3.713572,431.0,6.068426,100.000000,100.0,100.0,100.0,429.0,200.999985,GFP
394A_bc1_10_bc2_12_bc3_83_GFP_3,71,4.276666,755.0,6.628041,97.218543,100.0,100.0,100.0,747.0,201.000015,GFP
394A_bc1_10_bc2_15_bc3_43_none_5,6,1.945910,25.0,3.258096,100.000000,100.0,100.0,100.0,25.0,201.000000,none
394A_bc1_10_bc2_15_bc3_7_GFP_6,43,3.784190,469.0,6.152733,100.000000,100.0,100.0,100.0,468.0,201.000000,GFP
...,...,...,...,...,...,...,...,...,...,...,...
394A_bc1_9_bc2_95_bc3_37_none_29540,31,3.465736,232.0,5.451038,100.000000,100.0,100.0,100.0,232.0,201.000000,none
394A_bc1_9_bc2_95_bc3_43_GFP_29541,26,3.295837,355.0,5.874931,100.000000,100.0,100.0,100.0,353.0,201.000031,GFP
394A_bc1_9_bc2_9_bc3_36_RFP_29544,50,3.931826,454.0,6.120297,100.000000,100.0,100.0,100.0,453.0,201.000000,RFP
394A_bc1_9_bc2_9_bc3_50_GFP_29545,74,4.317488,971.0,6.879356,97.528321,100.0,100.0,100.0,970.0,201.000031,GFP


In [ ]:
import scipy

sc.pp.pca(dense2, n_comps = 100)
sc.pp.neighbors(dense2, n_pcs = 100, n_neighbors = 15)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


# Wishbone Starting Cell Identification and Trajectory

Runs Wishbone algorithm, separated anaylsis and running the algorithm in due to matplotlib issues

In [ ]:
import scanpy.external as sce

ecoli_blattman_copy = dense2.copy()
id = ecoli_blattman_copy.obs[:]['n_genes_by_counts'].idxmax()
print(id)

394A_bc1_45_bc2_41_bc3_84_RFP_12773


In [ ]:
ecoli_blattman_copy

AnnData object with n_obs × n_vars = 12255 × 1983
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'n_counts', 'cell_sum', 'express'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'mean', 'std'
    uns: 'log1p', 'pca', 'neighbors'
    obsm: 'X_pca'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [ ]:
sc.tl.tsne(adata=ecoli_blattman_copy, n_pcs=5, perplexity=30)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,


In [ ]:
sc.pp.neighbors(adata=ecoli_blattman_copy) # for some reason adding on other parameters makes wishbone not work with key errors
sc.tl.diffmap(adata=ecoli_blattman_copy, n_comps=10)

In [ ]:
state = True
while state:
  try:
    sce.tl.wishbone(adata=ecoli_blattman_copy, start_cell='394A_bc1_45_bc2_41_bc3_84_RFP_12773', components=[2, 3], num_waypoints=150, branch=True)
    state = False
  except KeyError:
    continue

Building lNN graph...
lNN computed in : 0.06 seconds
Determining waypoints if not specified...
Determining shortest path distances and perspectives....
Building lNN graph...
lNN computed in : 0.06 seconds
Determining waypoints if not specified...
Determining shortest path distances and perspectives....
Building lNN graph...
lNN computed in : 0.06 seconds
Determining waypoints if not specified...
Determining shortest path distances and perspectives....
Building lNN graph...
lNN computed in : 0.06 seconds
Determining waypoints if not specified...
Determining shortest path distances and perspectives....
Building lNN graph...
lNN computed in : 0.06 seconds
Determining waypoints if not specified...
Determining shortest path distances and perspectives....
......................................................................................................................................................
Time for determining distances and perspectives: 117.00 seconds
Determining branch point 

In [ ]:
sc.tl.umap(ecoli_blattman_copy)

In [ ]:
#sc.pl.tsne(ecoli_blattman_copy, color='trajectory_wishbone')

In [ ]:
#sc.pl.umap(ecoli_blattman_copy, color='branch_wishbone')

In [ ]:
#sc.pl.tsne(ecoli_blattman_copy, color='express')
#sc.pl.tsne(ecoli_blattman_copy, color=['zinT'])

Above confirms Blattman findings that "Of the 7,387 cells analysed, 61% did not contain any plasmid transcripts; their growth states were therefore ambiguous at first (grey points in PCA). However, using the PC1 threshold, we predicted that 92.2% of these were stationary-phase cells.Over-representation of stationary-phase cells in the ambiguous population was not surprising, as plasmid expression in stationary-phase cells was generally lower than in exponential-phase cells." 

also, trajectory moves from RFP to GFP

In [ ]:
ecoli_blattman_copy

In [ ]:
#sc.tl.rank_genes_groups(ecoli_blattman_copy, 'express', method='wilcoxon')
#sc.pl.rank_genes_groups(ecoli_blattman_copy)

In [ ]:
#markers_2 = 'oppABCDF','pdhR-aceEF-lpd'
#sce.pl.wishbone_marker_trajectory(ecoli_blattman_copy, markers_2)

# Differential Gene Expression

Output to file then coordinate with the "Wishbone Scanorama"

In [ ]:
sc.tl.leiden(ecoli_blattman_copy, resolution = 0.3)

In [ ]:
ecoli_blattman_copy.uns['leiden']

{'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 0.3}}

In [ ]:
!pip install scprep
import scprep

scprep.stats.differential_expression_by_cluster(ecoli_blattman_copy.X, ecoli_blattman_copy.obs['leiden'],gene_names = ecoli_blattman_copy.var_names)

{'0':                                              difference  rank
 pdhR-aceEF-lpd                                -0.633377     0
 rplNXE-rpsNH-rplFR-rpsE-rpmD-rplO-secY-rpmJ   -0.598518     1
 cyoABCDE                                      -0.581553     2
 rpsJ-rplCDWB-rpsS-rplV-rpsC-rplP-rpmC-rpsQ    -0.569070     3
 rpsMKD-rpoA-rplQ                              -0.519751     4
 ...                                                 ...   ...
 mdtNOP                                        -0.000056  1978
 yibF                                          -0.000056  1979
 greB                                          -0.000055  1980
 yaiI                                           0.000039  1981
 serV-argVYZQ                                   0.000039  1982
 
 [1983 rows x 2 columns],
 '1':                                              difference  rank
 rplNXE-rpsNH-rplFR-rpsE-rpmD-rplO-secY-rpmJ    1.033799     0
 pdhR-aceEF-lpd                                 1.024407     1
 rpsJ-rplCDWB-rps

In [ ]:
sc.tl.rank_genes_groups(ecoli_blattman_copy, 'leiden', method = 't-test', key_added = 't-test')
sc.pl.rank_genes_groups(ecoli_blattman_copy, n_genes=10, sharey=False, key = "t-test")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:421: RuntimeWarning: invalid value encountered in log2
  foldchanges[global_indices]


In [ ]:
sc.tl.rank_genes_groups(ecoli_blattman_copy, 'leiden', method='t-test_overestim_var', key_added = "t-test_ov")
sc.pl.rank_genes_groups(ecoli_blattman_copy, n_genes=10, sharey=False, key = "t-test_ov")

/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:421: RuntimeWarning: invalid value encountered in log2
  foldchanges[global_indices]


In [ ]:
sc.tl.rank_genes_groups(ecoli_blattman_copy, 'leiden', method='wilcoxon', key_added = "wilcoxon")
sc.pl.rank_genes_groups(ecoli_blattman_copy, n_genes=10, sharey=False, key="wilcoxon")

/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:421: RuntimeWarning: invalid value encountered in log2
  foldchanges[global_indices]


In [ ]:
sc.tl.rank_genes_groups(ecoli_blattman_copy, 'leiden', method='logreg',key_added = "logreg")
sc.pl.rank_genes_groups(ecoli_blattman_copy, n_genes=10, sharey=False, key = "logreg")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
sc.tl.dendrogram(ecoli_blattman_copy, groupby = 'leiden')
sc.pl.rank_genes_groups_heatmap(ecoli_blattman_copy, n_genes=5, key="wilcoxon", groupby="leiden", show_gene_labels=True)

In [ ]:
sc.pl.rank_genes_groups_dotplot(ecoli_blattman_copy, n_genes=5, key="wilcoxon", groupby="leiden")

In [ ]:
sc.pl.rank_genes_groups_stacked_violin(ecoli_blattman_copy, n_genes=5, key="wilcoxon", groupby="leiden")

In [ ]:
sc.pl.rank_genes_groups_matrixplot(ecoli_blattman_copy, n_genes=5, key="wilcoxon", groupby="leiden")

In [ ]:
ecoli_blattman_copy.write(filename = '/Colab Notebooks/Blattman Data Post TI')